# THIS DOCUMENT IS OUT OF DATE

You should instead refer to the Demes specification docs: https://popsim-consortium.github.io/demes-spec-docs/

# The Demes standard for writing demographic models

## Background

* Genetic simulators can simulate very complex demographic models.
* Describing a demographic model for input to a simulator can be tedious and error prone.
* There are almost as many input formats as there are simulators.


## What is Demes?

- The Demes Specification: a formal description of how a demographic model should be written,
    and how it should be interpreted by software.\
    https://popsim-consortium.github.io/demes-spec-docs/

- A Python library: also named `demes`.\
    https://popsim-consortium.github.io/demes-docs/


## An example Demes file

```yaml
# example_01.yaml
description: asymmetric migration between two extant demes
time_units: generations
defaults:
  epoch:
    start_size: 5000
demes:
  - id: X
    epochs:
      - end_time: 1000
  - id: A
    ancestors: [X]
  - id: B
    ancestors: [X]
migrations:
  - source: A
    dest: B
    rate: 1e-4
```

# The `demes` python library

## Installing with pip

```sh
python -m pip install demes
```

## Installing with conda

There is a known issue when using the conda base environment. Please use a specific named environment instead.

```sh
conda create -n demes "python=3.8"
conda activate demes
python -m pip install demes
```

## Installing msprime and stdpopsim

You'll also need to install `msprime` and `stdpopsim` to run a simulation.
E.g. `python -m install msprime stdpopsim` or `conda install msprime stdpopsim`.
See the [msprime installation docs](https://msprime.readthedocs.io/en/stable/installation.html) and/or the
[stdpopsim installation docs](https://stdpopsim.readthedocs.io/en/latest/installation.html) for more details.

## Loading a Demes graph from a YAML file

In [1]:
import demes

graph = demes.load("examples/example_01.yaml")

## Inspecting a Demes graph

The ancestor/descendant relationships between demes are represented as a graph (also known as a network).

In [2]:
graph

Graph(description='asymmetric migration between two extant demes', time_units='generations', generation_time=None, doi=[], demes=[Deme(id='X', description=None, start_time=inf, ancestors=[], proportions=[], epochs=[Epoch(start_time=inf, end_time=1000, start_size=5000, end_size=5000, size_function='constant', selfing_rate=0, cloning_rate=0)]), Deme(id='A', description=None, start_time=1000, ancestors=['X'], proportions=[1.0], epochs=[Epoch(start_time=1000, end_time=0, start_size=5000, end_size=5000, size_function='constant', selfing_rate=0, cloning_rate=0)]), Deme(id='B', description=None, start_time=1000, ancestors=['X'], proportions=[1.0], epochs=[Epoch(start_time=1000, end_time=0, start_size=5000, end_size=5000, size_function='constant', selfing_rate=0, cloning_rate=0)])], migrations=[AsymmetricMigration(start_time=1000, end_time=0, rate=0.0001, source='A', dest='B')], pulses=[])

In [3]:
for deme in graph.demes:
    print(f"{deme.id} has {len(deme.epochs)} epochs, and {len(deme.ancestors)} ancestors ({deme.ancestors})")

X has 1 epochs, and 0 ancestors ([])
A has 1 epochs, and 1 ancestors (['X'])
B has 1 epochs, and 1 ancestors (['X'])


In [4]:
print(graph.migrations)

[AsymmetricMigration(start_time=1000, end_time=0, rate=0.0001, source='A', dest='B')]


Further details regarding the `Graph`, and related classes, can be found in the [API Reference](https://popsim-consortium.github.io/demes-docs/main/api.html) section of the `demes` documentation.

## Simulating a Demes graph with msprime.
We can simulate using `msprime`. Note: the `demes.convert.to_msprime()` function used below will be removed from `demes` in the future, as msprime will soon include support for using demes `Graph` objects directly.

In [5]:
import demes.convert
import msprime

# Convert our demes.Graph into the data structures that msprime uses to describe a demographic model.
pc, de, mm = demes.convert.to_msprime(graph)

# Build a dictionary to translate from deme ID (names) to population indexes (used by msprime).
pop = {deme.id: i for i, deme in enumerate(graph.demes)}

# Msprime represents a sample as a tuple of (pop_index, time) pairs. The `samples` are a list of these tuples.
# Lets simulate 10 haplotypes from deme A at time 0, and 20 haplotypes from deme B at time 0.
samples = [(pop["A"], 0)] * 10 + [(pop["B"], 0)] * 20

# Simulate!
ts = msprime.simulate(
    population_configurations=pc,
    demographic_events=de,
    migration_matrix=mm,
    samples=samples,
    mutation_rate=1e-8,
    recombination_rate=1e-8,
    length=100000,  # 100 kb genomic region
)

The resulting tree sequence object, `ts`, can then be used following the `tskit` documentation. https://tskit.readthedocs.io/

In [6]:
ts.num_samples

30

In [7]:
ts.diversity()

array(0.00016966)

## Command line interface


The `demes` library also includes a minimal command line interface. Try running the following command to check that `examples/example_01.yaml` is a valid input file. If the file is valid, this command will produce no output.
```sh
python -m demes examples/example_01.yaml
```

This can also be run within the notebook:

In [8]:
%%bash
python -m demes examples/example_01.yaml

# Writing a Demes file

A Demes file is actually a [YAML](https://en.wikipedia.org/wiki/YAML) file. A YAML file contains `property: value` pairs, and Demes imposes restrictions by allowing only properties with specific names, that have specific types of values. But the details of YAML aren't really important, and we'll introduce the allowed properties gradually using examples.


## What is a deme?

A deme is a collection of individuals that share a set of population parameters at any given time. Consider the simplest population model: a single deme with constant population size.

![One deme, constant population size](fig/one-deme-constant-N.png)

This deme exists now (`0` generations ago), and also exists as far back in time as we're interested in looking (towards infinity generations ago). In Demes, we say that the deme has an infinite `start_time`.

### A single-deme constant-size Demes file

Note: in YAML, infinity is spelled `.inf`, not `inf`!

```yaml
# example_02.yaml
time_units: generations
demes:
  - id: A
    start_time: .inf
    epochs:
      - start_size: 1000
        end_time: 0
```

### Default values

Having a deme's `start_time` property be infinite is very common. So for demes without ancestors, the `start_time` may be omitted, and will default to `.inf`. Similarly, the `end_time` property of the final epoch may be omitted, and will default to `0`. The following file is equivalent.

```yaml
# example_03.yaml
time_units: generations
demes:
  - id: A
    epochs:
      - start_size: 1000
```

## What is an epoch?

We partition a deme's interval of existence into distinct `epochs`.  A deme always has at least one epoch. The population parameters for a deme are allowed to change over time, but are fixed within an epoch. Consider a single deme that suffered a bottleneck `50` generations ago.

![One deme, two epochs](fig/one-deme-two-epochs.png)

### A single-deme two-epoch Demes file

To specify a change in the population size, we've introduced a second epoch. We now need three time values to define the epoch boundaries: the `start_time` of the deme, the `end_time` of epoch 0, and the `end_time` of epoch 1. All epochs must be listed in time-descending order (from the past towards the present).

```yaml
# example_04.yaml
time_units: generations
demes:
  - id: B
    start_time: .inf
    epochs:
      - start_size: 1000
        end_time: 50
      - start_size: 200
        end_time: 0
```

The same example, using default values for the deme's `start_time` and the final epoch's `end_time`.

```yaml
# example_05.yaml
time_units: generations
demes:
  - id: B
    epochs:
      - start_size: 1000
        end_time: 50
      - start_size: 200
```

### Why don't epochs have a `start_time` too?

The start time of an epoch is inferred indirectly, by looking at the deme's `start_time` (for epoch 0), or by looking at the previous epoch's `end_time` (for epoch 1, 2, etc.).

## Exponential population size changes

In the previous example, the population size was constant in each epoch. But what if we wanted to model a period of exponential population size growth, or an exponential decay? In any given epoch, there are actually two population size parameters, `start_size` and `end_size`, which correspond to the size at the start and end of the epoch. If an epoch's `start_size` and `end_size` properties are not equal, then the epoch is defined to have an exponentially changing population size over the epoch's time interval. Two important implications of this system are:

* We don't need to specify an exponential rate parameter, as software can calculate this from the `start_size`, `end_size`, `start_time` and `end_time` values for an epoch.
* Infinitely long epochs must have a constant population size. So if a deme has an infinite `start_time`, then `start_size` and `end_size` must be equal for epoch 0.

### The ZigZag model from Schiffels & Durbin (2014)

```yaml
# example_06.yaml
description: A single population model with epochs of exponential growth and decay.
doi:
  - https://doi.org/10.1038/ng.3015
time_units: generations
demes:
  - id: generic
    epochs:
    - {end_time: 34133.31, start_size: 7156}
    - {end_time: 8533.33, end_size: 71560}
    - {end_time: 2133.33, end_size: 7156}
    - {end_time: 533.33, end_size: 71560}
    - {end_time: 133.33, end_size: 7156}
    - {end_time: 33.333, end_size: 71560}
    - {end_time: 0, end_size: 71560}
```

There are several new things in this example, so lets go through them one-by-one.

* The `doi` property is a list of strings corresponding to the DOI(s) for publication(s) in which the model was described. By convention, the elements of the `doi` list are URLs, but any string value can be used.
* A compact-form syntax is used for each epoch in the `epochs` list (with curly braces `{` and `}`), rather than using multiple lines.
* Epoch 0 has a `start_size`, but no `end_size`. Because this epoch has an infinite time span, the population size must be constant, so the epoch's `end_size` will be the same as the `start_size`.
* Epoch 1 has an `end_size`, but no `start_size`. So the `start_size` is inherited from the `end_size` of the previous epoch. So the `end_size` and `start_size` for this epoch are different, and there will be exponential population growth over the epoch.
* Epochs 2 through 5 also inherit their `start_size` from the previous epoch, and this is different from the `end_size` provided. Epochs 2 and 4 have exponential decay, whereas epochs 3 and 5 have exponential growth.
* The final epoch has a constant size.

## Multiple demes

### A split event

Suppose we're interested in modeling two demes, `A` and `B`. The two demes are related by a common ancestor, from which they split `1000` generations ago. In addition to `A` and `B`, we'll model their common ancestor as an additional deme, `X`. Here, we introduce the `ancestors` property of a deme, which is a list of deme IDs.

```yaml
# example_07.yaml
time_units: generations
demes:
  - id: X
    epochs:
      - end_time: 1000
        start_size: 2000
  - id: A
    ancestors:
      - X
    epochs:
      - start_size: 2000
  - id: B
    ancestors:
      - X
    epochs:
      - start_size: 2000
```

Note that lists can be written more compactly using square brackets (`[` and `]`, with a comma separating list items). By convention, we use the more compact form for the ancestors list. The following example is equivalent.

```yaml
# example_08.yaml
time_units: generations
demes:
  - id: X
    epochs:
      - end_time: 1000
        start_size: 2000
  - id: A
    ancestors: [X]
    epochs:
      - start_size: 2000
  - id: B
    ancestors: [X]
    epochs:
      - start_size: 2000
```

* When writing models involving ancestor/descendant relationships, ancestor demes must be listed before their descendents. Try moving deme `X` below one of the other demes and loading the file with `demes`.
* Demes `A` and `B` both have an ancestor, so their `start_time` does **not** default to `.inf`. In the special case above, their `start_time` is inherited from the `end_time` of their ancestor. I.e. both `A` and `B` have a `start_time` of `1000` generations ago.


### A branch event

An alternative way of modeling a population split, is for the ancestral deme to remain alive after the split. This could more correctly be described as a *branch* event, rather than split event. In the example below, deme `X` is still the ancestor of `A`, but `X` continues to exist until `0` generations ago (recall that `0` is the default value for the final epoch's `end_time`).

```yaml
# example_09.yaml
time_units: generations
demes:
  - id: X
    epochs:
      - start_size: 2000
  - id: A
    ancestors: [X]
    start_time: 1000
    epochs:
      - start_size: 2000
```

Now that `A`'s ancestor exists until `0` generations ago, we must specify `A`'s `start_time` explicitly. Try removing deme `A`'s `start_time` property, and loading the file with `demes`.

## Multiple ancestors

The `ancestors` property of a deme is a list, and yes, a deme can have multiple ancestors. But in addition, we now need to specify the proportion of ancestry inherited from each ancestor. This is done using a deme's `proportions` property.

```yaml
# example_10.yaml
time_units: generations
demes:
  - id: X
    epochs:
      - start_size: 2000
  - id: Y
    epochs:
      - start_size: 10000
  - id: A
    ancestors: [X, Y]
    proportions: [0.1, 0.9]
    start_time: 1000
    epochs:
      - start_size: 300
```

Try setting the `end_time` for demes `X` and `Y` and removing `A`'s `start_time`.

## Continuous migration

Let's again consider demes `A` and `B`, which are related via a common ancestor `X` (a split event).

### Asymmetric migration

We may define continuous migration from `A` to `B`. Concretely, we are modeling migrants born in deme `A`, the `source` deme, who (potentially) have offspring in deme `B`, the `dest` deme. 

```yaml
# example_11.yaml
time_units: generations
demes:
  - id: X
    epochs:
      - end_time: 1000
        start_size: 2000
  - id: A
    ancestors: [X]
    epochs:
      - start_size: 2000
  - id: B
    ancestors: [X]
    epochs:
      - start_size: 2000
migrations:
  - source: A
    dest: B
    rate: 1e-4
```

The migrations occur at a `rate` of `1e-4` per generation (regardless of the `time_units`!), and occur continuously over the lifetime of both `A` and `B`. In the example above, `A` and `B` have identical existence time intervals. What do you think will happen if we give `A` and `B` different `start_time` or `end_time` properties? What do you think will happen if the existence intervals of `A` and `B` don't overlap at all?

To obtain greater control over when migrations occur, we can use the `start_time` and `end_time` migration properties. Here we list multiple migrations

```yaml
# example_12.yaml
time_units: generations
demes:
  - id: X
    epochs:
      - end_time: 1000
        start_size: 2000
  - id: A
    ancestors: [X]
    epochs:
      - start_size: 2000
  - id: B
    ancestors: [X]
    epochs:
      - start_size: 2000
migrations:
  - {source: A, dest: B, rate: 1e-4, start_time: 100, end_time: 50}
  - {source: A, dest: B, rate: 1e-3, start_time: 50, end_time: 0}
  - {source: B, dest: A, rate: 1e-5, start_time: 10, end_time: 0}
```

What do you think will happen if we list two or more migrations with time intervals that overlap? Does it matter if the `source` and `dest` demes are the same in each case?

### Symmetric migration

It's common to model migrants in both directions simultaneously, with the same `rate`. We could use multiple asymmetric migrations, but it's simpler to specify a list of deme IDs (the migration's `demes` property), instead of a `source` and `dest`.

```yaml
# example_13.yaml
time_units: generations
demes:
  - id: X
    epochs:
      - end_time: 1000
        start_size: 2000
  - id: A
    ancestors: [X]
    epochs:
      - start_size: 2000
  - id: B
    ancestors: [X]
    epochs:
      - start_size: 2000
migrations:
  - demes: [A, B]
    rate: 1e-4
```

The `demes` property can list arbitrarily many demes. For example, we could define symmetric migration between all pairwise combinations of four demes.

```yaml
# example_14.yaml
time_units: generations
demes:
  - id: alpha
    epochs: 
      - start_size: 1000
  - id: beta
    epochs: 
      - start_size: 1000
  - id: gamma
    epochs: 
      - start_size: 1000
  - id: delta
    epochs: 
      - start_size: 1000
migrations:
  - demes: [alpha, beta, gamma, delta]
    rate: 1e-4
```

## A pulse of admixture

Sometimes, we want to model migration that is limited to a very short period of time. To model a pulse of migration that occurs over a single generation, we can define one or more `pulses`. The example below is very similar to the asymmetric migration example given previously, except a pulse has a `proportion` property instead of a rate, and the `time` of the pulse must also be specified.

```yaml
# example_15.yaml
time_units: generations
demes:
  - id: X
    epochs:
      - end_time: 1000
        start_size: 2000
  - id: A
    ancestors: [X]
    epochs:
      - start_size: 2000
  - id: B
    ancestors: [X]
    epochs:
      - start_size: 2000
pulses:
  - source: A
    dest: B
    proportion: 0.01
    time: 50
```

Note that an admixture pulse can also be modeled using multiple ancestors. Try doing this for the example above. What do you think are the advantages of one approach over the other?

What do you think will happen if we define multiple pulses at the same `time`? Does it matter whether the `source` and `dest` demes are the same in each pulse?

# Setting defaults

To avoid duplication in a Demes graph with many features, it's possible to set default values for some properties. Suppose we wish to define multiple demes, each with only one epoch, and each with a constant population size.

```yaml
# example_16.yaml
time_units: generations
defaults:
  # Note: this is spelled "epoch", as distinct from the "epochs" list.
  epoch:
    start_size: 1000
demes:
  - id: alpha
  - id: beta
  - id: gamma
  - id: delta
migrations:
  - demes: [alpha, beta, gamma, delta]
    rate: 1e-4
```

The epoch defaults can be overriden by providing an explicit value inside the desired epoch.

```yaml
# example_17.yaml
time_units: generations
defaults:
  epoch:
    start_size: 1000
demes:
  - id: alpha
  - id: beta
  - id: gamma
  - id: delta
    epochs:
      - end_time: 100
      - start_size: 200
        end_time: 0
migrations:
  - demes: [alpha, beta, gamma, delta]
    rate: 1e-4
```

It's also possible to provide defaults for properties of a deme, such as the `start_time` and `ancestors`.

```yaml
# example_18.yaml
time_units: generations
defaults:
  deme: {start_time: 1000, ancestors: [X]}
  epoch: {start_size: 1000}
demes:
  - id: X
    start_time: .inf
    ancestors: []
  - id: alpha
  - id: beta
  - id: gamma
  - id: delta
    epochs:
      - end_time: 100
      - start_size: 200
        end_time: 0
migrations:
  - demes: [alpha, beta, gamma, delta]
    rate: 1e-4
```

What do you think would happen if we didn't override the `start_time` and `ancestors` deme defaults for deme `X`?

Defaults for `migration` and `pulse` objects may also be specified for elements of the `migrations` and `pulses` lists.

# Time units and generation time

In the previous examples, we've exclusively set `time_units` to "generations". This is appropriate in many cases, but sometimes other units are more natural. For example, it's sometimes more natural to describe times using *years*, or even *thousands of years*. However, most simulation software operates using generations as the canonical unit of time. In Demes, the `time_units` property may be any string, but "generations" is special. If the `time_units` are **not** "generations", then an additional `generation_time` property must be specified. This latter property can then be used by the simulator to convert from the chosen `time_units` into units of generations.

**Warning:** The units for the `rate` of `migrations` are always per generation, even when the `time_units` are not generations. Only the `start_time`, `end_time`, and `time` properties should match the `time_units`.


```yaml
# example_19.yaml
description: The Gutenkunst et al. (2009) OOA model.
doi:
- https://doi.org/10.1371/journal.pgen.1000695
time_units: years
generation_time: 25

demes:
- id: ancestral
  description: Equilibrium/root population
  epochs:
  - {end_time: 220e3, start_size: 7300}
- id: AMH
  description: Anatomically modern humans
  ancestors: [ancestral]
  epochs:
  - {end_time: 140e3, start_size: 12300}
- id: OOA
  description: Bottleneck out-of-Africa population
  ancestors: [AMH]
  epochs:
  - {end_time: 21.2e3, start_size: 2100}
- id: YRI
  description: Yoruba in Ibadan, Nigeria
  ancestors: [AMH]
  epochs:
  - start_size: 12300
- id: CEU
  description: Utah Residents (CEPH) with Northern and Western European Ancestry
  ancestors: [OOA]
  epochs:
  - {start_size: 1000, end_size: 29725}
- id: CHB
  description: Han Chinese in Beijing, China
  ancestors: [OOA]
  epochs:
  - {start_size: 510, end_size: 54090}

migrations:
- {demes: [YRI, OOA], rate: 25e-5}
- {demes: [YRI, CEU], rate: 3e-5}
- {demes: [YRI, CHB], rate: 1.9e-5}
- {demes: [CEU, CHB], rate: 9.6e-5}
```